# AutoGluon

## Setup
### Correct the path

In [3]:
import sys
import os

# Get the current working directory
current_working_directory = os.getcwd()

# Go up one level from the current working directory
parent_directory = os.path.join(current_working_directory, '..')

# Add the parent directory to sys.path
sys.path.append(parent_directory)

os.getcwd()

'c:\\Users\\sverr\\OneDrive\\Desktop\\ai\\machine-learning-structure\\notebooks'

In [4]:
%pip install autogluon
%pip install scikit-learn

  Using cached autogluon-1.0.0-py3-none-any.whl (9.9 kB)
  Using cached autogluon.features-1.0.0-py3-none-any.whl (62 kB)
  Using cached autogluon.tabular-1.0.0-py3-none-any.whl (306 kB)
  Using cached autogluon.multimodal-1.0.0-py3-none-any.whl (416 kB)
  Using cached autogluon.core-1.0.0-py3-none-any.whl (229 kB)
  Using cached autogluon.timeseries-1.0.0-py3-none-any.whl (120 kB)
  Using cached pandas-2.1.4-cp310-cp310-win_amd64.whl (10.7 MB)
  Using cached autogluon.common-1.0.0-py3-none-any.whl (64 kB)
  Using cached boto3-1.34.70-py3-none-any.whl (139 kB)
  Using cached ray-2.6.3-cp310-cp310-win_amd64.whl (22.4 MB)
  Using cached hyperopt-0.2.7-py2.py3-none-any.whl (1.6 MB)
  Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)
  Using cached torch-2.0.1-cp310-cp310-win_amd64.whl (172.3 MB)
  Using cached timm-0.9.16-py3-none-any.whl (2.2 MB)
  Using cached scikit_image-0.20.0-cp310-cp310-win_amd64.whl (23.7 MB)
  Using cached torchmetrics-1.1.2-py3-none-any.whl (764 kB)
  U

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.7.0 requires pydantic>=2, but you have pydantic 1.10.14 which is incompatible.
You should consider upgrading via the 'c:\Users\sverr\OneDrive\Desktop\ai\machine-learning-structure\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\sverr\OneDrive\Desktop\ai\machine-learning-structure\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
%load_ext autoreload

In [6]:
%autoreload 

from sklearn.metrics import accuracy_score, classification_report
import autogluon.core as ag
from autogluon.tabular import TabularDataset, TabularPredictor

from src.features.post_processor import save_predictions
from src.features.ml_service import  prepare_data, prepare_test_data
from src.config import TARGET_FEATURES

c:\Users\sverr\OneDrive\Desktop\ai\machine-learning-structure\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load data

In [12]:
x_train, _, x_test, y_train, _, y_test = prepare_data(validation_size=0, test_size=0.1)
for target_feature_name in TARGET_FEATURES:
    x_train[target_feature_name] = y_train

data = TabularDataset(x_train)

InvalidParameterError: The 'test_size' parameter of train_test_split must be a float in the range (0.0, 1.0), an int in the range [1, inf) or None. Got 1.0 instead.

## Train model

In [8]:
# Initialize the AutoGluon TabularPredictor
predictor = TabularPredictor(label=target_feature_name).fit(train_data=data)

No path specified. Models will be saved in: "AutogluonModels\ag-20240325_224410"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20240325_224410"
=================== System

## Make predictions

In [9]:
# Evaluate on the test set
y_test_pred = predictor.predict(x_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy: ", test_accuracy)
print("Test Classification Report:\n", classification_report(y_test, y_test_pred))

Validation Accuracy:  0.929125
Test Accuracy:  0.930575
Test Classification Report:
               precision    recall  f1-score   support

           0       0.93      1.00      0.96     37217
           1       0.69      0.00      0.01      2783

    accuracy                           0.93     40000
   macro avg       0.81      0.50      0.49     40000
weighted avg       0.91      0.93      0.90     40000



## Save model

In [10]:
x_test = prepare_test_data()
final_predictions = predictor.predict(x_test)

In [11]:
# Save the final predictions as a CSV file
save_predictions(final_predictions, f'predictions_auto_gluon')